# Macros 

So far we have been creating Julia expressions ourselves, by hand. Macros are a common method to do this automatically.

Recall that macros begin with `@` and behave like "super-functions", which take in a piece of code and replace it with another piece of code.
The effect of a macro call is to splice, or replace, the new piece of code in place of the old code; the new code is what is actually compiled by the Julia compiler. 

Note that the user *does not need to explicitly pass an `Expr`ession object*; Julia turns the code that follows the macro call into an expression.

To see this, let's define the simplest macro:

In [1]:
macro simple(ex)
    @show ex
    @show typeof(ex)
    nothing   # return nothing for the moment
end

@simple (macro with 1 method)

and run it:

In [2]:
result = @simple a = b

ex = :(a = b)
typeof(ex) = Expr


We see that the code `a = b` has *automatically been captured as a Julia expression*. Basically the *string* `"a = b"` that we typed has been **parsed**, i.e. converted into Julia code. We can do this ourselves:

In [3]:
Meta.parse("a = b")

:(a = b)

The macro can now take this `Expr` object and process it as we have seen in the previous notebooks. The macro will return a new `Expr`; this is the new code that will actually be compiled instead of the old code!

## Structuring a macro

The usual recommendation is that a macro should just act as an interface to the user that captures the user's code, as we saw just above. The resulting expression is then usually passed to a function to do the manipulation. This gives a separation of concerns (capture vs. processing) and makes it easier for the developer to test the processing step.

For example, let's write a macro that replaces a `+` with a `*`:

In [4]:
macro add_to_mult(ex)
    return _add_to_mult(ex)
end

@add_to_mult (macro with 1 method)

The function receives an expression and should create the new expression:

In [5]:
function _add_to_mult(ex)
    ex.head == :call && ex.args[1] == :+ && (ex.args[1] = :*)    
    ex
end

_add_to_mult (generic function with 1 method)

In [6]:
_add_to_mult( :(a + b) )

:(a * b)

In [7]:
@add_to_mult a + b   # try to eval this code

LoadError: UndefVarError: a not defined

What is happening here? The macro *first* replaces the code with `a * b`, and *then* compiles and tries to execute the code. 
But the variable `a` does not yet exist, so this errors. Defining `a` and `b` behaves as we expect:

In [8]:
a = 2; b = 3

3

In [9]:
@add_to_mult a + b

6

We indeed get the result of multiplying `a` and `b`, not adding them.

This is a good example of why debugging macros is best done via an intermediate function that does the expression manipulation.

Recall that we can use `@macroexpand` to see what the macro is doing:

In [10]:
@macroexpand @add_to_mult a + b

:(Main.a * Main.b)

## Macro hygiene

Macro "hygiene" refers to the fact that macros do some modification of the code that they receive, in order to be "hygienic" (clean): they try not to touch user code.

## Exercise

1. Define a macro `@simple2` that returns the expression that was passed to it.


2. What happens when you call `@simple2 yy = xx^2`?


3. Define a variable `xx` with the value `3`. Does the macro work now?


4. Does the variable `yy` now exist?


5. To see what's happening, use `@macroexpand`.

In [11]:
macro simple2(expr)
    expr   # just return the expr as is
end

@simple2 (macro with 1 method)

In [12]:
xx = 10
@simple2 yy = xx^2

100

In [13]:
yy

LoadError: UndefVarError: yy not defined

In [14]:
@macroexpand @simple2 yy = xx^2

:(var"#32#yy" = Main.xx ^ 2)

You should find that the variable `yy` does *not* now exist, even though it seems like it should, since the code `yy = xx^2` was evaluated. However, macros by default do not "touch" variables in the context from where they are called, since this may have unintended consequences. We refer to this as macro **hygiene**: the macro is **hygienic**, i.e. clean, meaning that it does not "infect" the user's code.

Nonetheless, often we may *want* a macro to be able to modify a variable in the context from which the macro is called.
In which case we can "escape" from this hygiene, making a non-hygienic macro, using `esc` ("escape"):

In [15]:
macro simple3(ex)
    :($(esc(ex)))
end

@simple3 (macro with 1 method)

In [16]:
@simple3 yy = xx^2    ## now modifying yy

100

In [17]:
yy

100

In [18]:
@macroexpand @simple3 yy = xx^2

:(yy = xx ^ 2)

Note that once again the macro must return an *expression*. 

For code clarity it is possible to define a new variable that is the escaped version:

In [19]:
macro simple4(ex)
    ex2 = esc(ex)
    :($ex2)
end

@simple4 (macro with 1 method)

In [20]:
@macroexpand @simple4 yy = xx^2

:(yy = xx ^ 2)

In [21]:
xx = 3
@simple4 yy = 2^xx

8

## Exercise

1. Check that `@simple4` does create a variable `yy`.

In [22]:
yy

8

## Exercise

Define a `@myshow` macro that reproduces the behaviour of `@show`.

In [23]:
macro myshow(expr)
    # :(println(stdout, $(QuoteNode(expr)), " = ", $(esc(expr))))  
    local var = QuoteNode(expr)
    local val = esc(expr)
    :(println(stdout, $var, " => ", $(val)))  
end

@myshow (macro with 1 method)

In [24]:
@myshow x = 3

x = 3 => 3


In [25]:
x

3

## Exercise 8

1. Write a macro `@replace` that replaces terms in an expression. Apply it to `yy = xx^2 + xx`, replacing `xx` by `xx + 1`.


2. Write a macro `@checked` that replaces all arithmetic operations (`+`, `-`, `*`, `/`) with checked operations (`Base.checked_add` etc.)

# Macros for domain-specific languages

Let's see some simple examples of how we can start to approach domain-specific languages for scientific applications. 

Let's suppose we want to reproduce the `@variables` macro from `Symbolics.jl`. 
The idea is that there is a `Variable` object representing a symbolic variable:

In [26]:
struct Variable
    name::Symbol
end
# Base.show(io::IO, v::Variable) = print(io, v.name)

And we could define arithmetic operations on those to carry out symbolic manipulations (exercise!).

We can create such a variable as

In [27]:
Variable(:x)

Variable(:x)

To define a Julia variable called `x` that is bound to the `Variable` object, we must do

In [28]:
x = Variable(:x)

Variable(:x)

The situation is similar to the `@show` macro: we would ideally like to be able to write `@var x`, which expands to `x = Variable(:x)`.

## Exercise

1. Try to write the `@var` macro. You will probably get stuck! Where is the sticking point?

[Blank!]

In [29]:
macro var(expr)
    _var(expr)
end

function _var(expr)
    :($(esc(expr)) = Variable($(QuoteNode(expr))))
end

_var (generic function with 1 method)

In [30]:
@var x

Variable(:x)

In [31]:
# in more detail

function _var(expr)
    esc_expr = esc(expr)
    quoted_expr = QuoteNode(expr)
    # println(" => got esc_expr: ", esc_expr)
    # println(" => got quoted_expr: ", quoted_expr)
    :(
        $(esc_expr) = Variable($(quoted_expr))
    )
end

_var (generic function with 1 method)

In [32]:
_var(:x)

:($(Expr(:escape, :x)) = Variable(:x))

In [33]:
@macroexpand @var x

:(x = Main.Variable(:x))

In [34]:
@var z

Variable(:z)

In [35]:
z

Variable(:z)

In [36]:
macro var₂(expr)
    :($(esc(expr)) = Variable($(QuoteNode(expr)))) 
end

@var₂ (macro with 1 method)

In [37]:
@var₂ y

Variable(:y)

## Several variables

Now suppose we want to expand our macro to handle not only single variables, but also multiple variables, e.g. `@var x y`

The first task is to work out what happens when the macro receives this

In [38]:
macro simple(ex...)
    @show ex
    nothing
end

@simple (macro with 2 methods)

In [39]:
@simple x y

ex = (:x, :y)


In general we could have an arbitrary number of arguments; we should pass each through to the `_var` function:`

In [40]:
macro var(exprs...)
    all_code = [_var(expr) for expr ∈ exprs]
    @show all_code
    :nothing    
end

@var (macro with 2 methods)

In [41]:
@var x y

all_code = Expr[:($(Expr(:escape, :x)) = Variable(:x)), :($(Expr(:escape, :y)) = Variable(:y))]


Now we need to combine that code into a single code block:

In [42]:
macro var(exprs...)
    all_code = quote end  # empty block
    all_code.args = reduce(vcat, _var(expr) for expr ∈ exprs)
    # println("===> all_code: $(all_code) / typeof(all_code): $(typeof(all_code))")
    all_code.args
end

@var (macro with 2 methods)

In [43]:
res = @var x y z

3-element Vector{Any}:
 :($(Expr(:escape, :x)) = Variable(:x))
 :($(Expr(:escape, :y)) = Variable(:y))
 :($(Expr(:escape, :z)) = Variable(:z))

In [44]:
x, y, z

(Variable(:x), Variable(:y), Variable(:z))

In [45]:
res, typeof(res)

(Any[:($(Expr(:escape, :x)) = Variable(:x)), :($(Expr(:escape, :y)) = Variable(:y)), :($(Expr(:escape, :z)) = Variable(:z))], Vector{Any})

### Exercise: 

Rewrite the code so that a list of all the generated variables is returned.

In [50]:
w = 4; t = :x

:x

In [51]:
@var w t

2-element Vector{Any}:
 :($(Expr(:escape, :w)) = Variable(:w))
 :($(Expr(:escape, :t)) = Variable(:t))

In [52]:
t

:x

In [53]:
w

4

## Vectors of variables

Finally let's suppose we want to allow `@var x[1:10]` to make variables called `x1`, ..., `x10`.

The first thing to do is to work out how Julia stores `x[1:10]` as an expression:

In [54]:
Meta.parse("x[1:10]")

:(x[1:10])

In [55]:
dump(ans)

Expr
  head: Symbol ref
  args: Array{Any}((2,))
    1: Symbol x
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol :
        2: Int64 1
        3: Int64 10


We see that the answer is that the `head` of the `Expr` is `ref`.

## Exercise

1. Write a function to deal with this case and generate the variables

2. Modify the macro to do so

## Extra trick:

Thanks to Simeon Schaub for the following trick: 

You can call a macro in the following way to get back the expression:

In [56]:
var"@simple"(LineNumberNode(3), Main, x, y)

ex = (Variable(:x), Variable(:y))
